In [1]:
import pandas as pd
#from fuzzywuzzy import process

#### Step 0: Basic data exploration

In [2]:
df_fifa21 = pd.read_csv('../data/original/FIFA 21.csv')
df_fifa22 = pd.read_csv('../data/original/FIFA 22.csv')

print(f'There are {df_fifa21.shape[0]} players in FIFA 21')
print(f'There are {df_fifa22.shape[0]} players in FIFA 22')

if df_fifa21.shape[1] == df_fifa22.shape[1]:
    print(f'Each player is measured across {df_fifa21.shape[1]} dimensions in the video game') 

There are 19260 players in FIFA 21
There are 18539 players in FIFA 22
Each player is measured across 90 dimensions in the video game


Already we see that the data is high dimensional. This is because while designing the game, the makers have to ensure that every player has a rating in every single position. By design, gamers are allowed to assign any position to any player. 

Meaning a goalkeeper can be played as a striker and vice versa. Each goalkeeper therefore has a striker rating, and each striker has a goalkeeper rating. In real life, players stick to one singular position. Therefore, this data is just a good guess on how a plyer will perform in another position. 

This is not a feature that we need to include in our analysis. 

In [3]:
df_fifa21.head()

,ID,Name,FullName,Age,Height,Weight,PhotoUrl,Nationality,Overall,Potential,...,LMRating,CMRating,RMRating,LWBRating,CDMRating,RWBRating,LBRating,CBRating,RBRating,GKRating
0,158023,L. Messi,Lionel Messi,34,170,72,https://cdn.sofifa.com/players/158/023/22_60.png,Argentina,93,93,...,93,90,93,69,67,69,64,53,64,22
1,188545,R. Lewandowski,Robert Lewandowski,32,185,81,https://cdn.sofifa.com/players/188/545/22_60.png,Poland,92,92,...,87,83,87,67,69,67,64,63,64,22
2,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,36,187,83,https://cdn.sofifa.com/players/020/801/22_60.png,Portugal,91,91,...,89,81,89,66,62,66,63,56,63,23
3,231747,K. Mbappé,Kylian Mbappé,22,182,73,https://cdn.sofifa.com/players/231/747/22_60.png,France,91,95,...,92,84,92,70,66,70,66,57,66,21
4,200389,J. Oblak,Jan Oblak,28,188,87,https://cdn.sofifa.com/players/200/389/22_60.png,Slovenia,91,93,...,38,41,38,35,39,35,35,36,35,92


In [4]:
# This will have far fewer rows than the video game because the statistics are only measured for players who play in Europe (~2500)

df_stats21 = pd.read_csv('../data/original/Stats 21.csv', delimiter=';', encoding='latin1')
df_stats22 = pd.read_csv('../data/original/Stats 22.csv', delimiter=';', encoding='latin1')

print(f'There are {df_stats21.shape[0]} players whose statistics were measured in 2021')
print(f'There are {df_stats22.shape[0]} players whose statistics were measured in 2022')

if df_stats21.shape[1] == df_stats22.shape[1]:
    print(f'Each player has {df_stats21.shape[1]} parameters measured in real life') 
else:
    print(f"{df_stats21.shape[1]} parameters were measured in 2021")
    print(f"{df_stats22.shape[1]} parameters were measured in 2022")

There are 2921 players whose statistics were measured in 2021
There are 2689 players whose statistics were measured in 2022
143 parameters were measured in 2021
124 parameters were measured in 2022


Even though 20 fewer parameters were dropped, there are still way too many things being measured. I will limit myself to the 10 most representative parameters for both goalkeepers and outfield players for this project.

In [5]:
# Rename the stats Player column to fifa FullName column to enable merging 

df_stats21.rename(columns={'Player': 'FullName'}, inplace=True)
df_stats22.rename(columns={'Player': 'FullName'}, inplace=True)

df_stats21.head()

,Rk,FullName,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Off,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%
0,1,Max Aarons,ENG,DF,Norwich City,Premier League,22.0,2000,34,32,...,0.03,1.41,1.16,0.0,0.06,0.03,5.53,0.47,1.59,22.7
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34.0,1987,34,34,...,0.00,0.06,1.39,0.0,0.03,0.00,6.77,2.02,1.36,59.8
2,3,Salis Abdul Samed,GHA,MF,Clermont Foot,Ligue 1,22.0,2000,31,29,...,0.00,0.36,1.24,0.0,0.00,0.00,8.76,0.88,0.88,50.0
3,4,Laurent Abergel,FRA,MF,Lorient,Ligue 1,29.0,1993,34,34,...,0.03,0.79,2.23,0.0,0.00,0.00,8.87,0.43,0.43,50.0
4,5,Charles Abi,FRA,FW,Saint-Étienne,Ligue 1,22.0,2000,1,1,...,0.00,2.00,0.00,0.0,0.00,0.00,4.00,2.00,0.00,100.0


In [6]:
# Remove duplicates from fifa 
df_fifa21.drop_duplicates(inplace=True)
df_fifa22.drop_duplicates(inplace=True)

# Keep first instance of dupliacate in stats 
# Players can change clubs mid season, example: Ronaldo played for Juventus and Manchester United in 2021
df_stats21.drop_duplicates(subset='FullName', keep='first', inplace=True)
df_stats22.drop_duplicates(subset='FullName', keep='first', inplace=True)

🚨 Note on some limitations 

* In each dataset the names of clubs are slightly different. "Reims" in stats and "Stade de Reims" in fifa. With more time I could have fixed this but I will only be using the club mentioned in the fifa dataset. 
* Players can change clubs mid season, example: Ronaldo played for Juventus and Manchester United in 2021. Since I am storing each player's club season / year wise, I will not be able to store this information. I will only capture the first club that a player has played for in these instances. 

#### Step 1: Drop columns that aren't required


In [7]:
fifa_cols = ["ID", "FullName","Height", "Nationality", "Overall", "Potential", "BestPosition", "Club", "ValueEUR",
             "ContractUntil", "IntReputation", "PreferredFoot", "SkillMoves", "AttackingWorkRate", "DefensiveWorkRate",
             "PaceTotal", "ShootingTotal", "PassingTotal", "DribblingTotal", "DefendingTotal", "PhysicalityTotal", "GKDiving",
             "GKHandling", "GKKicking", "GKPositioning", "GKReflexes"]

df_fifa21 = df_fifa21[fifa_cols]
df_fifa22 = df_fifa22[fifa_cols]

In [8]:
stats_cols = ["FullName","Born","MP", "Starts", "Min", "90s", "Goals", "Assists", "PasTotCmp%", "PasProg", "GCA", "SCA",
              "Tkl+Int", "CarProg", "Clr", "TouDefPen", "AerWon%", "TouDef3rd", "PasLonCmp%", "Err","Recov"]

df_stats21 = df_stats21[stats_cols]
df_stats22 = df_stats22[stats_cols]

#### Step 2: Find players in the intersection of both fifa and statistics datasets. 

#### Then find players who exist across both years, else my model can make predictions about players found only in one year.

#### Output cleaned dataset. 

In [9]:
data_21 = pd.merge(df_fifa21, df_stats21, on='FullName', how='inner')

data_22 = pd.merge(df_fifa22, df_stats22, on='FullName', how='inner')   


🚨 Note on some limitations 

* The stats data uses inconsistent naming practices. This makes it harder to merge on the FullName. In the fifa data there are two seperate name columns called short name (initial. last name) and full name (first name middle name last name). In the stats data has some combination of this. This makes the matching tricky. 

* I tried to use fuzzy matching to do this, but that ended up messing up the player names and their ids. It ended in degraded data. Hence I chose to forgo names that aren't an exact match due to time  constraints and will only be working with about 60% of the data that was collected. 

In [36]:
# Find players that are found across both years, remove the ones that are not 

common_players = set(data_21['ID']).intersection(set(data_22['ID']))

# Filter both datasets to keep only common players
data_21 = data_21[data_21['ID'].isin(common_players)]
data_22 = data_22[data_22['ID'].isin(common_players)]

data_21 = data_21.sort_values(by='ID').reset_index(drop=True)
data_22 = data_22.sort_values(by='ID').reset_index(drop=True)

if data_21['ID'].equals(data_22['ID']):
    print(f'There are {data_21.shape[0]} players that are found across both years')

# Store output in csv files 
data_21.to_csv('../data/cleaned/Data 21.csv', index=False)
data_22.to_csv('../data/cleaned/Data 22.csv', index=False)

There are 1145 players that are found across both years
